In [11]:
import pandas as pd
import json
from streamlit_folium import folium_static

In [8]:
def load_data():
    # getting region data in geojson
    map_pref_region = pd.read_csv('../data/wikidf_pref.csv').drop(columns="Unnamed: 0")[['Prefecture.1', 'Region']].set_index('Prefecture.1')
    geojson_pref = '../data/jp_prefs.geojson'
    with open(geojson_pref, "r") as file:
        jsonData = json.load(file)
    for pref in jsonData['features']:
        pref['properties']['REGION'] = map_pref_region.loc[pref['properties']['NAME_JP'], 'Region']

    # prefecture data
    df_prefecture = pd.read_csv('../data/prefecture_list_no_outliers.csv')
    dict_pref_names = {}
    for pref in jsonData['features']:
        dict_pref_names[pref['properties']['NAME_JP']] = pref['properties']['NAME']
    df_prefecture['prefecture_eng2'] = df_prefecture.prefecture.map(dict_pref_names)

    # region data
    df_region = df_prefecture.groupby('region').agg(
        num_sake = ('num_sake', 'sum'),
        num_company = ('num_company', 'sum'),
        abv_avg = ('abv_avg', 'mean'),
        abv_count = ('abv_count', 'sum'),
        abv_std = ('abv_avg', 'std'),
        acidity_avg = ('acidity_avg', 'mean'),
        acidity_count = ('acidity_count', 'sum'),
        acidity_std = ('acidity_avg', 'std'),
        gravity_avg = ('gravity_avg', 'mean'),
        gravity_count = ('gravity_count', 'sum'),
        gravity_std = ('gravity_avg', 'std'),
    )
    dict_region_acidity = pd.Series(df_region['acidity_avg'].values, index=df_region.index).to_dict()
    dict_region_gravity = pd.Series(df_region['gravity_avg'].values, index=df_region.index).to_dict()

    # adding regional acidity and gravity to prefecture data
    df_prefecture['average_acidity'] = df_prefecture.region.map(dict_region_acidity)
    df_prefecture['average_gravity'] = df_prefecture.region.map(dict_region_gravity)

    return df_prefecture


In [9]:
df_prefecture = load_data()

In [10]:
df_prefecture.head()

,prefecture,prefecture_eng,region,num_sake,num_company,abv_avg,abv_count,abv_std,acidity_avg,acidity_count,acidity_std,gravity_avg,gravity_count,gravity_std,prefecture_eng2,average_acidity,average_gravity
0,三重県,Mie,Kansai,388,25,15.860544,294,1.379991,1.526538,130,0.268659,1.943781,201,3.551109,Mie,1.511923,2.067919
1,京都府,Kyōto,Kansai,746,35,15.652660,658,1.640044,1.440302,248,0.331750,1.690113,354,4.459346,Kyoto,1.511923,2.067919
2,佐賀県,Saga,Kyūshū,458,24,15.979297,384,1.167905,1.598469,196,0.312143,0.864087,252,4.842799,Saga,1.485090,1.776175
3,兵庫県,Hyōgo,Kansai,1013,54,15.685358,823,1.769784,1.437654,405,0.235460,2.079167,564,4.105231,Hyogo,1.511923,2.067919
4,北海道,Hokkaido,Hokkaidō,220,10,15.435000,200,1.285862,1.347863,117,0.168972,3.068182,132,3.599074,Hokkaido,1.347863,3.068182


In [7]:
pd.read_csv('../data/prefecture_list_no_outliers.csv').head()

,prefecture,prefecture_eng,region,num_sake,num_company,abv_avg,abv_count,abv_std,acidity_avg,acidity_count,acidity_std,gravity_avg,gravity_count,gravity_std
0,三重県,Mie,Kansai,388,25,15.860544,294,1.379991,1.526538,130,0.268659,1.943781,201,3.551109
1,京都府,Kyōto,Kansai,746,35,15.652660,658,1.640044,1.440302,248,0.331750,1.690113,354,4.459346
2,佐賀県,Saga,Kyūshū,458,24,15.979297,384,1.167905,1.598469,196,0.312143,0.864087,252,4.842799
3,兵庫県,Hyōgo,Kansai,1013,54,15.685358,823,1.769784,1.437654,405,0.235460,2.079167,564,4.105231
4,北海道,Hokkaido,Hokkaidō,220,10,15.435000,200,1.285862,1.347863,117,0.168972,3.068182,132,3.599074
